

#  Titanic  Predictions 

(This is a good and simple code for you)  

**Please consider upvoting if this is useful to you! :)**

**Contents 
**
1-Import Necessary Libraries

2-Read In and Explore the Data

3-Data Analysis

4-Data Visualization

5-Cleaning Data

6-Building a Logistic Regression model

7-Train Test Split

8-Evaluation

# 1) Import Necessary Libraries

**1-1: data analysis libraries**

In [ ]:
import numpy as np
import pandas as pd

**1-2: visualization libraries**

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import missingno
%matplotlib inline

In [ ]:
import cufflinks as cf
cf.go_offline()

**2-Read our training and testing data**

**Importing our CSV files**

In [ ]:
my_train_data = pd.read_csv("../input/titanic/train.csv")
my_test_data = pd.read_csv("../input/titanic/test.csv")

# Let's have a look at the datasets

**Data types, missing data and summary statistics**

Looking training data by describe() and info()

In [ ]:
my_train_data.describe(include="all")

# Nacessary Information (according to this table)


Survival: Survival (0 = No; 1 = Yes)

Pclass: Passenger Class (1 = 1st; 2 = 2nd; 3 = 3rd)

Name : Name

Sex : Sex

Age : Age

Sibsp : Number of Siblings/Spouses Aboard

Parch : Number of Parents/Children Aboard

Ticket : Ticket Number

Fare : Passenger Fare

Cabin : Cabin Number

Embarked : Port of Embarkation (C = Cherbourg; Q = Queenstown; S = Southampton)

In [ ]:
print("Training set shape is: ", my_train_data.shape)
print("Test set shape is: ", my_test_data.shape)

In [ ]:
my_train_data.info()

# 3-Data Analysis

whats up in our dataset

get a list of the features within the titanic dataset

In [ ]:
print(my_train_data.columns)

In [ ]:
my_train_data.head(7)

In [ ]:
my_train_data.info()
print('-'*40)
my_test_data.info()

**let go and watch useless values**

In [ ]:
#Missing data in training set
pd.isnull(my_train_data).sum().sort_values(ascending = False)

* We can see that except for the abovementioned missing values, no NaN values exist.

In [ ]:
missingno.matrix(my_train_data)

In [ ]:
#Missing data in testing set
my_test_data.isnull().sum().sort_values(ascending = False)

In [ ]:
missingno.matrix(my_test_data)

# * **Categorical variables**

# A) SEX

Some Questions!

How many females survived the Titanic?

What was the survival rate on the Titanic?

In [ ]:
print (my_train_data[["Sex", "Survived"]].groupby(['Sex'], as_index= False).mean().sort_values(by='Survived', ascending = False))

# B) Pclass

Some Questions!

Did anyone from 3rd class survived the Titanic?

In [ ]:
my_train_data['Pclass'].value_counts()

In [ ]:
my_train_data[['Pclass', 'Survived']].groupby(['Pclass'], as_index = False).mean().sort_values(by = 'Survived', ascending = False)

# C) Embarked

In [ ]:
my_train_data['Embarked'].value_counts(dropna = False)
# Comment: Two missing values

In [ ]:
my_train_data[['Embarked', 'Survived']].groupby(['Embarked'], as_index = False).mean().sort_values(by = 'Survived', ascending = False)

# **4-Data Visualization**

**What was the survival rate on the Titanic(by Sex and Pclass)**

Survival Rate based on Sex and Pclass(passenger class (1st, 2nd, 3rd))

In [ ]:
sns.barplot(x="Sex", y="Pclass", data=my_train_data,palette="Set3")

In [ ]:
sns.countplot(x="Sex", data= my_train_data, palette="Dark2")

In [ ]:
sns.barplot(x="Sex", y="Survived", data=my_train_data, palette="hls")

**This chart shows survival of passangers on the titanic(by Sex)**

In [ ]:
sns.barplot(x = 'Pclass', y ='Survived', data = my_train_data)
plt.ylabel('Survival Probability')
plt.title('Survival Probability by Passenger Class')

In [ ]:
se = sns.factorplot(x = 'Pclass', y = 'Survived', hue = 'Sex', data = my_train_data, kind = 'bar')
se.despine(left = False)
plt.ylabel('Survival Probability')
plt.title('Survival Probability by Sex and Passenger Class')

In [ ]:
sns.set_style('whitegrid')
sns.countplot(x='Survived',hue='Sex',data=my_train_data,palette='RdBu_r')

In [ ]:
sns.set_style('whitegrid')
sns.countplot(x='Survived',hue='Pclass',data=my_train_data,palette='rainbow')

**This box show Percentage of females who survived by using train data**

In [ ]:
print("Percentage of females who Survived is=", my_train_data["Survived"][my_train_data["Sex"] == 'female'].value_counts(normalize = True)[1]*100)

* Number of passengers saved by age

In [ ]:
sns.distplot(my_train_data['Age'].dropna(),kde=False,color='darkred',bins=30)

In [ ]:
my_train_data['Age'].hist(bins=30,color='darkred',alpha=0.7)

* Number of passengers saved by SibSp

In [ ]:
sns.countplot(x='SibSp',data=my_train_data)

In [ ]:
print("Percentage of males who survived is=", my_train_data["Survived"][my_train_data["Sex"] == 'male'].value_counts(normalize = True)[1]*100)

*  Ticket Prices for the Titanic

In [ ]:
my_train_data['Fare'].hist(color='green',bins=40,figsize=(8,4))

# Cufflinks:
 
Cufflinks is another library that connects the Pandas data frame with Plotly enabling users to create visualizations directly from Pandas

**Let's take a quick moment to show an example of cufflinks!**

In [ ]:
my_train_data['Fare'].iplot(kind='hist',bins=30,color='green')

**Data Cleaning**

* We want to fill in missing age data instead of just dropping the missing age data rows.

* One way to do this is by filling in the mean age of all the passengers (imputation). 

* However we can be smarter about this and check the average age by passenger class. For example:

In [ ]:
plt.figure(figsize=(12, 7))
sns.boxplot(x='Pclass',y='Age',data=my_train_data,palette='winter')


We can see the wealthier passengers in the higher classes tend to be older,which makes sense.

We'll use these average age values to impute based on Pclass for Age.

In [ ]:
def impute_age(cols):
    Age = cols[0]
    Pclass = cols[1]
    
    if pd.isnull(Age):

        if Pclass == 1:
            return 37

        elif Pclass == 2:
            return 29

        else:
            return 24

    else:
        return Age

**Now apply that function!**

In [ ]:
my_train_data['Age'] = my_train_data[['Age','Pclass']].apply(impute_age,axis=1)

**Now let's check that heat map again!**

In [ ]:
sns.heatmap(my_train_data.isnull(),yticklabels=False,cbar=False,cmap='viridis')

**Great! Let's go ahead and drop the Cabin column and the row in Embarked that is NaN.**

In [ ]:
my_train_data.drop('Cabin',axis=1,inplace=True)

In [ ]:
my_train_data.head()

In [ ]:
my_train_data.dropna(inplace=False)

**Converting Categorical Features:**

We'll need to convert categorical features to dummy variables using pandas! Otherwise our machine learning algorithm won't be able to directly take in those features as inputs.

In [ ]:
my_train_data.info()

In [ ]:
sex = pd.get_dummies(my_train_data['Sex'],drop_first=True)
embark = pd.get_dummies(my_train_data['Embarked'],drop_first=True)

In [ ]:
my_train_data.drop(['Sex','Embarked','Name','Ticket'],axis=1,inplace=True)

In [ ]:
my_train_data = pd.concat([my_train_data,sex,embark],axis=1)

In [ ]:
my_train_data.head()

**Great! Our data is ready for our model!
**
# Building a Logistic Regression model
 
Let's start by splitting our data into a training set and test set.
# Train Test Split

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(my_train_data.drop('Survived',axis=1), 
                                                    my_train_data['Survived'], test_size=0.30, 
                                                    random_state=101)

# Training and Predicting

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
logmodel = LogisticRegression()
logmodel.fit(X_train,y_train)

In [ ]:
predictions = logmodel.predict(X_test)


**Let's move on to evaluate our model!**

# Evaluation

We can check precision,recall,f1-score using classification report!

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
print(classification_report(y_test,predictions))

****Please consider upvoting if this is useful to you!****